In [44]:
from google.cloud import aiplatform

job = aiplatform.CustomContainerTrainingJob(
    display_name="train-penguin-model",
    command=[
        "penguin-model", 
        "bigquery-public-data.ml_datasets.penguins"
    ],
    container_uri="europe-west1-docker.pkg.dev/cde-ds-enablement-8k1r/vertex/penguin_model",
    staging_bucket="gs://cde-dse-penguin-artifacts",
    location="europe-west1",
    model_serving_container_image_uri="europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
)

job

In [45]:
job.run(
    model_display_name="penguin-model",
    service_account="pipeline-penguin@cde-ds-enablement-8k1r.iam.gserviceaccount.com"
)

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://cde-dse-penguin-artifacts/aiplatform-custom-training-2021-11-08-13:21:02.142 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west1/training/5078459490743877632?project=409590257761
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/409590257761/locations/europe-west1/trainingPipelines/5078459490743877632 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/europe-west1/training/2229932726432038912?project=409590257761
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/409590257761/locations/europe-west1/trainingPipelines/5078459490743877632 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob proje

KeyboardInterrupt: 

In [12]:
job.run?

Signature:
job.run(
    dataset: Union[google.cloud.aiplatform.datasets.image_dataset.ImageDataset, google.cloud.aiplatform.datasets.tabular_dataset.TabularDataset, google.cloud.aiplatform.datasets.text_dataset.TextDataset, google.cloud.aiplatform.datasets.video_dataset.VideoDataset, NoneType] = None,
    annotation_schema_uri: Union[str, NoneType] = None,
    model_display_name: Union[str, NoneType] = None,
    model_labels: Union[Dict[str, str], NoneType] = None,
    base_output_dir: Union[str, NoneType] = None,
    service_account: Union[str, NoneType] = None,
    network: Union[str, NoneType] = None,
    bigquery_destination: Union[str, NoneType] = None,
    args: Union[List[Union[float, int, str]], NoneType] = None,
    environment_variables: Union[Dict[str, str], NoneType] = None,
    replica_count: int = 1,
    machine_type: str = 'n1-standard-4',
    accelerator_type: str = 'ACCELERATOR_TYPE_UNSPECIFIED',
    accelerator_count: int = 0,
    boot_disk_type: str = 'pd-ssd',
    b

In [132]:
from typing import Optional, NamedTuple

import kfp
from kfp.v2.google import experimental
import kfp.components as comp
from kfp.v2 import compiler  # noqa: F811
from kfp.v2.dsl import (
    component,
    Input,
    InputPath,
    OutputPath,
    Output,
    Dataset,
    Metrics,
)


@component(
    base_image="python:3.9-slim",
    packages_to_install=["google-cloud-bigquery", "pandas", "pyarrow"],
    output_component_file="query.yaml"
)
def query(
    query: str, output_path: Dataset["Parquet"], project_id: Optional[str] = None
) -> None:
    """Calculates sum of two arguments"""

    from google.cloud import bigquery

    client = bigquery.Client(project=project_id)
    job = client.query(query)

    df = job.to_dataframe()
    df.to_parquet(output_path)


# query = comp.create_component_from_func(
#     _query,
#     base_image="python:3.9-slim",
#     packages_to_install=["google-cloud-bigquery", "pandas", "pyarrow"],
#     output_component_file="query.yaml",
# )


train_model = comp.load_component_from_text(
    """
name: Train model
description: Trains our model

inputs:
- {name: train_dataset, type: Parquet, description: 'Train dataset'}

outputs:
- {name: model, type: Model, description: 'Output model'}

implementation:
  container:
    image: europe-west1-docker.pkg.dev/cde-ds-enablement-8k1r/vertex/penguin_model
    command: [
      penguin-model, 
      {inputPath: train_dataset},
      {outputPath: model}
    ]
"""
)

@component(
    base_image="python:3.9-slim",
    output_component_file="eval_model.yaml"
)
def eval_model(model_path: InputPath("Model"), metrics: Output[Metrics]) -> NamedTuple(
    'EvalModelOutput',
    [
      ('roc', float)
    ]):
  print(model_path)

  metrics.log_metric("roc", 0.9)
  

@kfp.dsl.pipeline(name="penguin")
def pipeline():

    query_task = query(
        "SELECT * FROM bigquery-public-data.ml_datasets.penguins",
        project_id="cde-ds-enablement-8k1r",
    )

    train_task = (
        train_model(query_task.outputs["output_path"])
        # TODO: Figure out how to set constaints in the v2 API.
        # Docs: https://www.kubeflow.org/docs/distributions/gke/pipelines/enable-gpu-and-tpu/
        # .set_gpu_limit(1).add_node_selector_constraint(
        #     "cloud.google.com/gke-accelerator", "nvidia-tesla-k80"
        # )
    )

    eval_task = eval_model(train_task.outputs["model"])

    # model_upload_op = gcc_aip.ModelUploadOp(
    #     project=project,
    #     display_name=model_display_name,
    #     artifact_uri=WORKING_DIR,
    #     serving_container_image_uri=serving_container_image_uri,
    #     # serving_container_environment_variables={"NOT_USED": "NO_VALUE"},
    # )
    # model_upload_op.after(train_task)


compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="pipeline.json",
)


TypeError: 'type' object is not subscriptable

In [129]:
from google.cloud.aiplatform.pipeline_jobs import PipelineJob

job = PipelineJob(
    display_name="penguins",
    enable_caching=False,
    template_path="pipeline.json",
    parameter_values={},
    pipeline_root="gs://cde-dse-penguin-artifacts/pipelines",
    location="europe-west1",
)

job.run(service_account="pipeline-penguin@cde-ds-enablement-8k1r.iam.gserviceaccount.com")

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/409590257761/locations/europe-west1/pipelineJobs/penguin-20211108161424
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/409590257761/locations/europe-west1/pipelineJobs/penguin-20211108161424')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/europe-west1/pipelines/runs/penguin-20211108161424?project=409590257761
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/409590257761/locations/europe-west1/pipelineJobs/penguin-20211108161424 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/409590257761/locations/europe-west1/pipelineJobs/penguin-20211108161424 current s

KeyboardInterrupt: 

In [ ]:
job.run(
    model_display_name="penguin-model",
    service_account="pipeline-penguin@cde-ds-enablement-8k1r.iam.gserviceaccount.com"
)

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://cde-dse-penguin-artifacts/aiplatform-custom-training-2021-11-08-13:21:02.142 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west1/training/5078459490743877632?project=409590257761
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/409590257761/locations/europe-west1/trainingPipelines/5078459490743877632 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/europe-west1/training/2229932726432038912?project=409590257761
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/409590257761/locations/europe-west1/trainingPipelines/5078459490743877632 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob proje

KeyboardInterrupt: 

In [58]:
PipelineJob?

Init signature:
PipelineJob(
    display_name: str,
    template_path: str,
    job_id: Union[str, NoneType] = None,
    pipeline_root: Union[str, NoneType] = None,
    parameter_values: Union[Dict[str, Any], NoneType] = None,
    enable_caching: Union[bool, NoneType] = None,
    encryption_spec_key_name: Union[str, NoneType] = None,
    labels: Union[Dict[str, str], NoneType] = None,
    credentials: Union[google.auth.credentials.Credentials, NoneType] = None,
    project: Union[str, NoneType] = None,
    location: Union[str, NoneType] = None,
)
Docstring:     
Allows optional asynchronous calls to this Vertex AI Resource
Nouns.
Init docstring:
Retrieves a PipelineJob resource and instantiates its
representation.

Args:
    display_name (str):
        Required. The user-defined name of this Pipeline.
    template_path (str):
        Required. The path of PipelineJob or PipelineSpec JSON file. It
        can be a local path or a Google Cloud Storage URI.
        Example: "gs://project.

In [22]:
experimental.run_as_aiplatform_custom_job?

Signature:
experimental.run_as_aiplatform_custom_job(
    op: kfp.dsl._container_op.ContainerOp,
    display_name: Union[str, NoneType] = None,
    replica_count: Union[int, NoneType] = None,
    machine_type: Union[str, NoneType] = None,
    accelerator_type: Union[str, NoneType] = None,
    accelerator_count: Union[int, NoneType] = None,
    boot_disk_type: Union[str, NoneType] = None,
    boot_disk_size_gb: Union[int, NoneType] = None,
    timeout: Union[str, NoneType] = None,
    restart_job_on_worker_restart: Union[bool, NoneType] = None,
    service_account: Union[str, NoneType] = None,
    network: Union[str, NoneType] = None,
    output_uri_prefix: Union[str, NoneType] = None,
    worker_pool_specs: Union[List[Mapping[str, Any]], NoneType] = None,
) -> None
Docstring:
Run a pipeline task using AI Platform (Unified) custom training job.

For detailed doc of the service, please refer to
https://cloud.google.com/ai-platform-unified/docs/training/create-custom-job

Args:
  op: The 